In [2]:
import os

In [3]:
%pwd

'd:\\Chicken_Disease_ENd_to_End\\research'

In [4]:
os.chdir("../")

In [16]:
%pwd

'd:\\Chicken_Disease_ENd_to_End'

In [17]:
from dataclasses import dataclass
from pathlib import Path

In [18]:
@dataclass(frozen=True)
class BaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [19]:
from ChickenDisease.constants import *
from ChickenDisease.utils.common import read_yaml, create_directories

In [20]:
read_yaml(CONFIG_FILE_PATH).prepare_base_model

[2023-07-03 22:12:20,739: INFO: common: yaml file: config\config.yaml loaded successfully]


ConfigBox({'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'})

In [21]:
create_directories([read_yaml(CONFIG_FILE_PATH).prepare_base_model.root_dir])

[2023-07-03 22:12:21,069: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-03 22:12:21,079: INFO: common: created directory at: artifacts/prepare_base_model]


In [22]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_base_model_config(self) ->BaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])
        
        base_model_config = BaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
            
        )
        return base_model_config
        
        

In [23]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [24]:
class PrepareBaseModel:
    def __init__(self, config:BaseModelConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = self.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params,
            include_top=self.config.params_include_top
        )
        self.save_model(path=self.config.base_model_path, model=self.model)
        
    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
                
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation='softmax',
        )(flatten_in)
        
        full_model = tf.keras.layers.Model(
            inputs=model.input,
            outputs=prediction
        )
        
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
        )
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
        
    @staticmethod
    def save_model(path:Path, model: tf.keras.Model):
        model.save(path)

In [25]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_base_model_config()
    prepare_base_model = PrepareBaseModel(config=base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e
    

[2023-07-03 22:12:22,450: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-03 22:12:22,466: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-03 22:12:22,466: INFO: common: created directory at: artifacts]
[2023-07-03 22:12:22,474: INFO: common: created directory at: artifacts/prepare_base_model]


BoxKeyError: "'ConfigBox' object has no attribute 'classes'"